In [31]:
import redshift_connector
import keyring
import tabulate
import tkinter as tk
from tkinter import simpledialog

In [32]:
def set_username():
    # Prompt the user for the username in a pop-up window
    root = tk.Tk()
    root.withdraw()  # Hide the root window
    db_username = simpledialog.askstring("Username", "Enter your username", show='*')

    if db_username:
        # Store the username securely using keyring
        service = "db"  # Define a name for the service (can be anything)
        credential = "username"  # Specify the credential you're saving
        keyring.set_password(service, credential, db_username)
        print("Username saved securely!")

set_username()

Username saved securely!


In [33]:
def set_password():
    # Prompt the user for the password in a pop-up window
    root = tk.Tk()
    root.withdraw()  # Hide the root window
    db_password = simpledialog.askstring("Password", "Enter your password", show='*')

    if db_password:
        # Store the password securely using keyring
        service = "db"  # Define a name for the service (can be anything)
        credential = "password"  # Specify the credential you're saving
        keyring.set_password(service, credential, db_password)
        print("Password saved securely!")

set_password()

Password saved securely!


In [34]:
db_username = keyring.get_password("db", "username")
db_password = keyring.get_password("db", "password")

In [35]:
con = redshift_connector.connect(
    host = "ohdsi-lab-redshift-cluster-prod.clsyktjhufn7.us-east-1.redshift.amazonaws.com",
    database = "ohdsi_lab",
    port = 5439,
    user = db_username,
    password = db_password
)

In [36]:
cursor = con.cursor()

In [37]:
query = ("""select distinct 
	co.person_id,
	(select min(co_sub.condition_start_date)
	 from omop_cdm_synpuf_110k_531.condition_occurrence co_sub
	 inner join omop_cdm_synpuf_110k_531.concept_ancestor ca_sub
	 	on co_sub.condition_concept_id = ca_sub.descendant_concept_id
	 where ca_sub.ancestor_concept_id = 201826
	 and co_sub.person_id = co.person_id) as cohort_start_date,
	 op.observation_period_end_date as cohort_end_date
from omop_cdm_synpuf_110k_531.condition_occurrence co
inner join omop_cdm_synpuf_110k_531.concept_ancestor ca
	on co.condition_concept_id = ca.descendant_concept_id 
inner join omop_cdm_synpuf_110k_531.drug_exposure de
	on co.person_id = de.person_id 
inner join omop_cdm_synpuf_110k_531.observation_period op
	on co.person_id = op.person_id 
where ca.ancestor_concept_id = 201826
and de.drug_concept_id = 40164929
and de.drug_exposure_start_date 
	between co.condition_start_date
		and dateadd("day", 30, co.condition_start_date)
order by co.person_id;""")

In [38]:
cursor.execute(query)
result = cursor.fetchall()
columns = [desc[0] for desc in cursor.description]
print(tabulate.tabulate(result, headers = columns, tablefmt = "grid"))

+-------------+---------------------+-------------------+
|   person_id | cohort_start_date   | cohort_end_date   |
+=============+=====================+===================+
|          25 | 2008-03-22          | 2010-12-01        |
+-------------+---------------------+-------------------+
|          82 | 2008-03-12          | 2010-06-05        |
+-------------+---------------------+-------------------+
|          86 | 2008-03-06          | 2010-07-29        |
+-------------+---------------------+-------------------+
|          90 | 2008-06-10          | 2010-11-07        |
+-------------+---------------------+-------------------+
|          95 | 2008-02-24          | 2010-11-13        |
+-------------+---------------------+-------------------+
|         104 | 2008-01-19          | 2010-12-22        |
+-------------+---------------------+-------------------+
|         108 | 2008-02-11          | 2010-12-30        |
+-------------+---------------------+-------------------+
|         149 